In [ ]:
## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Torbay,47.6666,-52.7314,78.76,63,76,3.74,CA,broken clouds
1,1,Ushuaia,-54.8000,-68.3000,31.66,86,40,17.27,AR,shower snow
2,2,Orlik,52.5178,99.8250,47.80,85,85,2.15,RU,overcast clouds
3,3,Bambous Virieux,-20.3428,57.7575,69.22,69,94,20.11,MU,light rain
4,4,Puerto Ayora,-0.7393,-90.3518,78.76,86,73,5.01,EC,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new 
# DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Torbay,47.6666,-52.7314,78.76,63,76,3.74,CA,broken clouds
4,4,Puerto Ayora,-0.7393,-90.3518,78.76,86,73,5.01,EC,broken clouds
11,11,Sinnamary,5.3833,-52.9500,81.63,76,20,8.70,GF,few clouds
20,20,Pescara,42.3333,13.9500,82.56,37,2,4.88,IT,clear sky
23,23,Kaseda,31.4167,130.3167,78.64,85,76,13.06,JP,broken clouds
25,25,Hobyo,5.3505,48.5268,76.15,82,35,24.56,SO,scattered clouds
26,26,Victoria,22.2855,114.1577,86.18,80,0,4.00,HK,clear sky
27,27,Merritt,50.1123,-120.7942,84.25,22,54,8.48,CA,broken clouds
30,30,Georgetown,5.4112,100.3354,80.53,78,20,0.00,MY,few clouds
31,31,Benicarlo,40.4165,0.4271,82.90,75,0,4.05,ES,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                251
City                   251
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Country                251
Current Description    251
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

251


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Torbay,47.6666,-52.7314,78.76,63,76,3.74,CA,broken clouds
4,4,Puerto Ayora,-0.7393,-90.3518,78.76,86,73,5.01,EC,broken clouds
11,11,Sinnamary,5.3833,-52.9500,81.63,76,20,8.70,GF,few clouds
20,20,Pescara,42.3333,13.9500,82.56,37,2,4.88,IT,clear sky
23,23,Kaseda,31.4167,130.3167,78.64,85,76,13.06,JP,broken clouds


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Torbay,CA,78.76,broken clouds,47.6666,-52.7314,
4,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
11,Sinnamary,GF,81.63,few clouds,5.3833,-52.9500,
20,Pescara,IT,82.56,clear sky,42.3333,13.9500,
23,Kaseda,JP,78.64,broken clouds,31.4167,130.3167,
25,Hobyo,SO,76.15,scattered clouds,5.3505,48.5268,
26,Victoria,HK,86.18,clear sky,22.2855,114.1577,
27,Merritt,CA,84.25,broken clouds,50.1123,-120.7942,
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
31,Benicarlo,ES,82.90,clear sky,40.4165,0.4271,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Torbay,CA,78.76,broken clouds,47.6666,-52.7314,Gallows Cove
4,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Sinnamary,GF,81.63,few clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
20,Pescara,IT,82.56,clear sky,42.3333,13.9500,Casa Gavenniae
23,Kaseda,JP,78.64,broken clouds,31.4167,130.3167,Sun Stay Kaseda
25,Hobyo,SO,76.15,scattered clouds,5.3505,48.5268,Iftin hotel hobyo
26,Victoria,HK,86.18,clear sky,22.2855,114.1577,Mini Central
27,Merritt,CA,84.25,broken clouds,50.1123,-120.7942,Ramada Limited Merritt
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
31,Benicarlo,ES,82.90,clear sky,40.4165,0.4271,Hotel & Spa Peñíscola Plaza Suites


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))